# Tools definition

First, we'll create tools that actually work to do what we want here.

In [1]:
lyrics = {
    "title": "Fading Away",
    "intro": [
        "In the silence, I hear your voice",
        "A whispered reminder of our choice",
        "To let go, to move on, to lose",
        "The love we had, the memories we chose"
    ],
    "verse_one": [
        "We said forever, but forever's come and gone",
        "And now I'm left to face this empty dawn",
        "Your smile, a distant memory, a fleeting thought",
        "I'm searching for a way to heal my heart, to make it stop"
    ],
    "pre_chorus": [
        "But the pain remains, like an open sore",
        "A constant reminder of what we had before",
        "I'm trying to move on, but I'm stuck in this place",
        "Where love and loss collide, and my heart can't escape"
    ],
    "chorus": [
        "And I'm fading away, like a ghost in the night",
        "My heart is breaking, but it's not feeling right",
        "I'm trying to hold on, but you're slipping through my hands",
        "Leaving me with nothing but these tears and this pain"
    ],
    "verse_two": [
        "We said forever, but forever's just a lie",
        "And now I'm left to wonder why",
        "You walked away, without a fight",
        "Leaving me here, to face the dark of night"
    ],
    "outro": [
        "And I'll keep on fading, into the shadows of my mind",
        "Where love and loss entwine, and my heart will forever be left behind",
        "But maybe someday, I'll learn to let go",
        "And find my way back home, where love will set me free"
    ]
}

Let's start with the chorus.

In [2]:
lyrics['chorus']

["And I'm fading away, like a ghost in the night",
 "My heart is breaking, but it's not feeling right",
 "I'm trying to hold on, but you're slipping through my hands",
 'Leaving me with nothing but these tears and this pain']

In [3]:
import os
os.chdir('..')

In [83]:
from typing import Dict
from smolagents import tool
from utils.syllable_split import get_syllable_dict
import re, string

def get_words(verse):
    s = verse
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    verse_no_punct = regex.sub('', s)
    preprocessed_verse = verse_no_punct.lower().strip()
    words = preprocessed_verse.split()
    return words


def process_syllables(word, phonemes):
    preprocessed_phonemes = [
        re.sub('\d', '', re.sub(' ', '', p)) for p in phonemes
    ]
    cut_points = [
        p[0] for p in preprocessed_phonemes[1:]
    ]
    final_word = ''
    cut_word = word
    for cut in cut_points:
        if cut in cut_word:
            splitted = re.split(cut, cut_word)
            rest = [cut + s for s in splitted[1:]]
            cut_word = ''.join(rest)
            final_word += splitted[0] + '-'
    final_word += cut_word
    return final_word


@tool
def divide_syllables(verse: str, syllable_dict: Dict[str, str]) -> Dict[str, int]:
    """
    Divides a verse into syllables.

    Args:
        verse: A verse of a song.
        syllable_dict: the dictionary that maps words to its syllables.
    
    Returns:
        A dictionary with the verse text divided into syllables and syllable count.
    """
    words = get_words(verse)
    phonemes = [syllable_dict.get(word, '') for word in words]
    syllables = ''
    
    for i in range(len(words)):
        if len(phonemes[i]) == 1:
            syllables += words[i] + ' '
        else:
            syllables += process_syllables(words[i], phonemes[i]) + ' '
    syllables = syllables.strip()
    syllables_count = syllables.count(' ') + syllables.count('-') + 1
    return {
        'text': syllables,
        'syllables': syllables_count
    }

<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Lalasinha\AppData\Local\Temp\ipykernel_11052\439697072.py:17: SyntaxWarning: invalid escape sequence '\d'
  re.sub('\d', '', re.sub(' ', '', p)) for p in phonemes


In [87]:
syllable_dict = get_syllable_dict()
verse_syllables = []

for verse in lyrics['chorus']:
    verse_syllables.append(divide_syllables(verse, syllable_dict))

verse_syllables

[{'text': 'and im fa-ding a-way like a ghost in the night', 'syllables': 12},
 {'text': 'my heart is brea-king but its not fee-ling right', 'syllables': 11},
 {'text': 'im try-ing to hold on but youre sli-pping through my hands',
  'syllables': 13},
 {'text': 'lea-ving me with no-thing but these tears and this pain',
  'syllables': 12}]

In [136]:
import numpy as np

def get_mean_syllables_per_stanza(stanza):
    syllable_dict = get_syllable_dict()

    stanza_syllables = []
    for verse in stanza:
        stanza_syllables.append(
            divide_syllables(verse, syllable_dict)
        )
    
    syllables_in_stanza = [
        verse['syllables'] for verse in stanza_syllables
    ]

    mean_syllables_per_stanza = np.round(
        np.mean(syllables_in_stanza)
    )

    return mean_syllables_per_stanza


def get_possible_time_signatures(stanza):
    mean_syllables = get_mean_syllables_per_stanza(stanza)
    signatures = {
        2: "2/4",
        3: "3/4",
        4: "4/4",
        6: "6/8",
        9: "9/8",
        12: "12/8"
    }

    answer = []
    for k in signatures.keys():
        if mean_syllables%k == 0:
            answer.append(signatures[k])
    
    return answer


def get_number_of_compasses(stanza, time_signature):
    mean_syllables = get_mean_syllables_per_stanza(stanza)
    verses = len(stanza)
    number_of_notes, type_of_notes = time_signature.split('/')
    note_type = int(int(type_of_notes)/4)
    compasses = round(mean_syllables/float(number_of_notes))
    
    print(compasses, 'compasses per verse')
    print(number_of_notes, '1/'+str(note_type), 'notes per compass')
    print(verses*compasses, 'compasses in total')

In [138]:
get_number_of_compasses(lyrics['chorus'], '4/4')

3 compasses per verse
4 1/1 notes per compass
12 compasses in total
